In [1]:
import pandas as pd
import tidydata
import warnings

In [6]:
#  выгрузим список индикаторв
indicators = tidydata.indicators_scrapper(first_ind = '58536', last_ind = '58468')

In [ ]:
#  выгрузим файлы данных с сайта федстата
tidydata.fedstat_files_loader(indicators, executable_path)

In [2]:
#  преобразуем данные
#warnings.filterwarnings('ignore')
data_dir = '/home/dad/projects/tgu/test_task/data/' #  путь к папке с файлами данных
c1 = tidydata.combiner_1(data_dir)
c2 = tidydata.combiner_8(data_dir)
c3 = tidydata.combiner_9(data_dir)
c4 = tidydata.combiner_12(data_dir)
c5 = tidydata.combiner_51(data_dir)
c6 = tidydata.combiner_01(data_dir)
c7 = tidydata.combiner_04(data_dir)
c8 = tidydata.combiner_11(data_dir)
c9 = tidydata.combiner_02(data_dir)
c10 = tidydata.combiner_24(data_dir)
c11 = tidydata.combiner_26(data_dir)
c12 = tidydata.combiner_30(data_dir)

/home/dad/projects/tgu/test_task/tidydata.py:546: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_df[1][mask] = '_T'
/home/dad/projects/tgu/test_task/tidydata.py:554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_df[1][mask] = '_T'
/home/dad/anaconda3/envs/tgu/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/d

In [3]:
result = pd.concat([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12], axis=0)     #  соединим промежуточные результаты в одну таблицу
result['Значение'] = result['Значение'].apply( lambda x: ','.join(str(x).split('.')) ) # замена разделитля десятичной части дроби на запятую
result.dropna(subset=['Значение'], inplace=True) #  отфильтруем пропущенные значения
filter = result['Значение'].isin([0, 'Null'])    #  отфильтруем нулевые значения
result[~filter].to_csv('tidydata.csv', encoding='utf-8', sep=';', decimal=',', index = False) #  сохраним результаты в файл csv

CPU times: user 421 ms, sys: 16.1 ms, total: 437 ms
Wall time: 437 ms

In [6]:
result

,Расшифровка,Обозреваемый период,Единица измерения,Единица физических величин,Значение,Дата обновления данных,Пол,Возраст,Тип населенного пункта,Частота предоставления данных,Регион,Номер файла,Раздел,ссылка
2,Доля валовой добавленной стоимости туристской ...,2011,PT,PT,3,14.10.2020,_T,_T,_T,A,Российская Федерация,3,_T,https://fedstat.ru/indicator/58683
3,Доля валовой добавленной стоимости туристской ...,2012,PT,PT,3,14.10.2020,_T,_T,_T,A,Российская Федерация,3,_T,https://fedstat.ru/indicator/58683
4,Доля валовой добавленной стоимости туристской ...,2013,PT,PT,"3,2",14.10.2020,_T,_T,_T,A,Российская Федерация,3,_T,https://fedstat.ru/indicator/58683
5,Доля валовой добавленной стоимости туристской ...,2014,PT,PT,"3,4",14.10.2020,_T,_T,_T,A,Российская Федерация,3,_T,https://fedstat.ru/indicator/58683
6,Доля валовой добавленной стоимости туристской ...,2015,PT,PT,"3,3",14.10.2020,_T,_T,_T,A,Российская Федерация,3,_T,https://fedstat.ru/indicator/58683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,"Заболеваемость туберкулезом (3.3.2) (человек, ...",2015,человек,человек,"104,8",31.08.2021,Мужчины,Всего,_T,A,Российская Федерация,30,_T,https://www.fedstat.ru/organizations/indicator...
21,"Заболеваемость туберкулезом (3.3.2) (человек, ...",2015,человек,человек,85,31.08.2021,Мужчины,Всего,_T,A,Российская Федерация,30,_T,https://www.fedstat.ru/organizations/indicator...
22,"Заболеваемость туберкулезом (3.3.2) (человек, ...",2016,человек,человек,"12,5",31.08.2021,Мужчины,Всего,_T,A,Российская Федерация,30,_T,https://www.fedstat.ru/organizations/indicator...
22,"Заболеваемость туберкулезом (3.3.2) (человек, ...",2016,человек,человек,"96,4",31.08.2021,Мужчины,Всего,_T,A,Российская Федерация,30,_T,https://www.fedstat.ru/organizations/indicator...
